# Assignment 3 #

We are given three columns of integers with a row for each node. The first two columns contain x and y coordinates of the node positions in a plane. The third column contains node costs. The goal is to select exactly 50% of the nodes (if the number of nodes is odd we round the number of nodes to be selected up) and form a Hamiltonian cycle (closed path) through this set of nodes such that the sum of the total length of the path plus the total cost of the selected nodes is minimized. The distances between nodes are calculated as Euclidean distances rounded mathematically to integer values. The distance matrix should be calculated just after reading an instance and then only the distance matrix (no nodes coordinates) should be accessed by optimization methods to allow instances defined only by distance matrices. 

## Read the data ##

In [157]:
import pandas as pd
import numpy as np
from numpy.typing import ArrayLike, NDArray
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from math import sqrt

In [158]:
# read data into dataframes
instances = {
    "A": pd.read_csv("data/TSPA.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "B": pd.read_csv("data/TSPB.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "C": pd.read_csv("data/TSPC.csv", sep=';', header=None, names=["x", "y", "cost"]),
    "D": pd.read_csv("data/TSPD.csv", sep=';', header=None, names=["x", "y", "cost"]),
}

In [159]:
len(instances["A"])

200

In [160]:
def calculate_distance_matrix(df: pd.DataFrame) -> NDArray[np.int32]:
    """
    Calculate the distance matrix from the dataframe.
    The dataframe contains 'x' and 'y' columns for the coordinates.
    The distances are Euclidean, rounded to the nearest integer + the cost of the destination node.
    """
    coordinates = df[['x', 'y']].to_numpy()
    costs = df["cost"].to_numpy()
    dist_matrix = np.zeros(shape=(len(df), len(df)))
    for i in range(len(coordinates)):
        for j in range(len(coordinates)):
            dist_matrix[i, j] = round(sqrt((coordinates[i, 0] - coordinates[j, 0])**2 + (coordinates[i, 1] - coordinates[j, 1])**2)) + costs[j]
    return dist_matrix

In [161]:
distances_matrices = {
    "A": calculate_distance_matrix(instances["A"]),
    "B": calculate_distance_matrix(instances["B"]),
    "C": calculate_distance_matrix(instances["C"]),
    "D": calculate_distance_matrix(instances["D"])
}

costs = {
    "A": instances["A"]["cost"].to_numpy(),
    "B": instances["B"]["cost"].to_numpy(),
    "C": instances["C"]["cost"].to_numpy(),
    "D": instances["D"]["cost"].to_numpy()
}

In [162]:
def visualize_selected_route(
    selected_nodes_indices: ArrayLike, 
    dataframe: pd.DataFrame,
    title: str) -> None:
    """
    Visualize the selected route returned by the algorithm, including the cost of each node represented by a colormap.

    Parameters:
    selected_nodes_indices (list): Indices of the selected nodes in the route.
    dataframe (DataFrame): DataFrame containing 'x', 'y', and 'cost' columns for each node.
    """
    x = dataframe.loc[selected_nodes_indices, 'x']
    y = dataframe.loc[selected_nodes_indices, 'y']
    costs = dataframe.loc[selected_nodes_indices, 'cost']

    cmap = plt.cm.get_cmap('viridis')
    norm = plt.Normalize(vmin=min(costs), vmax=max(costs))

    plt.figure(figsize=(15, 10))
    scatter = plt.scatter(x, y, c=costs, cmap=cmap, norm=norm, s=100)
    plt.colorbar(scatter, label='Node Cost')

    for i, node in enumerate(selected_nodes_indices):
        start_node = selected_nodes_indices[i]
        end_node = selected_nodes_indices[(i + 1) % len(selected_nodes_indices)]
        plt.plot([x[start_node], x[end_node]], [y[start_node], y[end_node]], 'k-', lw=1)

    plt.title(title, fontsize=18)
    plt.xlabel('X Coordinate', fontsize=14)
    plt.ylabel('Y Coordinate', fontsize=14)
    plt.grid(True)
    plt.show()

In [163]:

def inter_route_exchange(selected , unselected, score,dist_matrix):
    solutions = []
    for x in range(len(selected)+1):
        for node in unselected:
            temp_score = score
            #wklejenie unselected node pomiedzy selected
            temp = selected[:x]+[node]+selected[x:]
            #usunięcie i zmienienie edga poprzedzającego node
            temp_score -= dist_matrix[selected[x-1]][selected[x]]
            temp_score += dist_matrix[selected[x-1]][node]

            #usunięcie i zmienienie edga następującego po node
            #jeśli index x+1 byłoby out of range ubezpieczenie
            if selected[x]==selected[-1]:
                idx = selected[0]
            else:
                idx = selected[x+1]
            temp_score -= dist_matrix[selected[x]][idx]
            temp_score += dist_matrix[node][idx]
            
            solutions.append((temp,temp_score))
            
    return solutions

def two_edges_exchange(selected, score, dist_matrix):
    solutions=[]
    for x in range(len(selected)-1): 
        for y in range(x+1,len(selected)):
            temp = selected[:]
            temp_score = score

            # jeśli y+1 jest idx out of range
            if temp[y] == temp[-1]:
                idy=temp[0]
            else:
                idy=temp[y+1]

            #część do odjęcia od dystansów, cześć rozwiązania do odwócenia plus sąsiadujące nody które się nie zmieniają
            part_to_reverse = [temp[x-1]] + temp[x:y+1] + [idy]


            #część do dodania do dystansów, część rozwiązania odwrócona plus sąsiadujące nody które się nie zmieniają 
            reversed_part = [temp[x-1]] + temp[x:y+1][::-1] + [idy]


            # usunięcie i zmienienie dystansów części któraj jest odwrócona
            for idx in range(len(part_to_reverse)):
                temp_score -= dist_matrix[part_to_reverse[idx-1]][part_to_reverse[idx]]
                temp_score += dist_matrix[reversed_part[idx-1]][reversed_part[idx]]
            
            temp[x:y+1] = temp[x:y+1][::-1]
            
            solutions.append((temp, temp_score))
    return solutions

def two_nodes_exchange(selected, score, dist_matrix):
    solutions=[]
    for x in range(len(selected)): 
        for y in range(x+1,len(selected)):
            temp=selected[:]
            temp_score=score
            #podmiana nodów
            temp[x],temp[y]=temp[y],temp[x]

            #usunięcie i zmienienie edgy poprzedzających node
            temp_score-=dist_matrix[temp[x-1]][temp[x]]
            temp_score+=dist_matrix[temp[x-1]][temp[y]]
            temp_score-=dist_matrix[temp[y-1]][temp[y]]
            temp_score+=dist_matrix[temp[y-1]][temp[x]]

            #kiedy byłby index x+1 out of range
            if temp[-1]==temp[x]:
                idx=temp[0]
            else: 
                idx=temp[x+1]

            #kiedy byłby index y+1 out of range 
            if temp[-1]==temp[y]:
                idy=temp[0]
            else:
                idy=temp[y+1]
            
            #usunięcie i zmienienie edgy następujących po node
            temp_score-=dist_matrix[temp[x]][idx]
            temp_score-=dist_matrix[temp[y]][idy]
            temp_score+=dist_matrix[temp[x]][idy]
            temp_score+=dist_matrix[temp[y]][idx]


            solutions.append((temp,temp_score))
    return solutions

def objective_function(solution,dist_matrix):
    for x in range(len(solution)):
        total_dist+=dist_matrix[solution[x-1],solution[x]]
    return total_dist

In [164]:
l=[x for x in range(10)]
z=[x for x in range(10,13)]
two_edges_exchange(l,2000,distances_matrices["A"])

[([1, 0, 2, 3, 4, 5, 6, 7, 8, 9], 1135.0),
 ([2, 1, 0, 3, 4, 5, 6, 7, 8, 9], 779.0),
 ([3, 2, 1, 0, 4, 5, 6, 7, 8, 9], 1976.0),
 ([4, 3, 2, 1, 0, 5, 6, 7, 8, 9], 1919.0),
 ([5, 4, 3, 2, 1, 0, 6, 7, 8, 9], 2640.0),
 ([6, 5, 4, 3, 2, 1, 0, 7, 8, 9], 1687.0),
 ([7, 6, 5, 4, 3, 2, 1, 0, 8, 9], 2230.0),
 ([8, 7, 6, 5, 4, 3, 2, 1, 0, 9], 2000.0),
 ([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], -606.0),
 ([0, 2, 1, 3, 4, 5, 6, 7, 8, 9], 1999.0),
 ([0, 3, 2, 1, 4, 5, 6, 7, 8, 9], 783.0),
 ([0, 4, 3, 2, 1, 5, 6, 7, 8, 9], 919.0),
 ([0, 5, 4, 3, 2, 1, 6, 7, 8, 9], 3557.0),
 ([0, 6, 5, 4, 3, 2, 1, 7, 8, 9], 3396.0),
 ([0, 7, 6, 5, 4, 3, 2, 1, 8, 9], 241.0),
 ([0, 8, 7, 6, 5, 4, 3, 2, 1, 9], 2492.0),
 ([0, 9, 8, 7, 6, 5, 4, 3, 2, 1], 2000.0),
 ([0, 1, 3, 2, 4, 5, 6, 7, 8, 9], 2031.0),
 ([0, 1, 4, 3, 2, 5, 6, 7, 8, 9], 1976.0),
 ([0, 1, 5, 4, 3, 2, 6, 7, 8, 9], 3283.0),
 ([0, 1, 6, 5, 4, 3, 2, 7, 8, 9], 3125.0),
 ([0, 1, 7, 6, 5, 4, 3, 2, 8, 9], 2607.0),
 ([0, 1, 8, 7, 6, 5, 4, 3, 2, 9], 1939.0),
 ([0, 1, 9, 8, 